In [1]:
import pandas as pd
from utils.prompter import Prompter
prompt_template_name = "alpaca"

data = pd.read_csv('tweets_raw_2021.csv')
data = data[['full_text', 'day']]

prompter = Prompter(prompt_template_name)
instructions = []
for idx,row in data.iterrows():
    full_text = row['full_text']
    ins = 'What is the sentiment toward Bitcoin in the input sentence? [positive, negative, neutral]'
    full_prompt = prompter.generate_prompt(
            ins,
            full_text,
        )
    instructions.append(full_prompt)
print(instructions[0])




Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is the sentiment toward Bitcoin in the input sentence? [positive, negative, neutral]

### Input:
What was your best trade of 2020?

### Response:



In [2]:
import os
import sys
from typing import List


import torch
import transformers
from datasets import load_dataset

"""
Unused imports:
import torch.nn as nn
import bitsandbytes as bnb
"""

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer

from utils.prompter import Prompter
# lora hyperparams
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_target_modules = [
    "q_proj",
    "v_proj",
]

prompt_template_name = "alpaca"  # The prompt template to use, will default to alpaca.
device_map = "auto"

base_model = "/root/llama-7b-hf"  # the only required argument
output_dir = "/root/autodl-tmp/output"
lora_weights = "/root/autodl-tmp/checkpoint-5800"
# resume_from_checkpoint = "/root/autodl-tmp/output/checkpoint-3700"
resume_from_checkpoint=None


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Shanghai')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/root/LLM-finetune/Untitled.ipynb')}
  warn(msg)


In [3]:
tokenizer = LlamaTokenizer.from_pretrained(base_model)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
 )
tokenizer.padding_side = "left"  # Allow batched inference

In [4]:
model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map=device_map,
    )
model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16,
)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
import json
from torch.utils.data import DataLoader
import time
import torch.nn.functional as F
from tqdm import tqdm

data['pred'] = ''
data['scores'] = ''
data['prob'] = ''
start_time = time.time()
model.eval()
with torch.no_grad():
    batch_size = 16
    input_texts = []
    targets = []
    # for idx, item in enumerate(instructions[300:]):
    #     # feature = format_example(item)
    #     # input_text = feature["context"]
    #     all = all + 1
    #     input_texts.append(item["context"])
    #     targets.append(item["target"])
    test_loader = DataLoader(instructions, batch_size=batch_size)
    for batch_idx,batch in enumerate(tqdm(test_loader)):
        input_ids = tokenizer(batch, padding=True,return_tensors='pt').to('cuda')
        # out = model.generate(**input_ids, max_length=250, temperature=0)
        out = model.generate(
            **input_ids,
            temperature=0,
            return_dict_in_generate= True,
            output_scores=True,
            max_new_tokens = 1
        )
        seqs = out['sequences']
        scores = out['scores'][0]
        softmax = F.softmax(scores, dim=1)
        
        pos = scores[:,1066]
        neg = scores[:,22198]
        neu = scores[:,17821]
        
        pos_prob = softmax[:,1066]
        neg_prob = softmax[:,22198]
        neu_prob = softmax[:,17821]

        max_val, max_index = torch.max(softmax, dim=1)
        
        # print(max_val, max_index)
#         print(tokenizer.decode(max_index))
        
        
        results = tokenizer.batch_decode(seqs)
        # print(results, '\n\n')
        # break
        for idx,res in enumerate(results):
            # print(res,'\n\n')
            pred = res[res.find('Response') + 10:]
            score_str = f'pos={pos[idx].item()}\nneg={neg[idx].item()}\nneu={neu[idx].item()}\n'
            prob_str = f'pos={pos_prob[idx].item()}\nneg={neg_prob[idx].item()}\nneu={neu_prob[idx].item()}\n'
            # print(batch_idx*batch_size + idx,prob_str,pred)
            data.at[batch_idx*batch_size + idx, 'pred'] = pred
            data.at[batch_idx*batch_size + idx, 'scores'] = score_str
            data.at[batch_idx*batch_size + idx, 'prob'] = prob_str
        data.to_csv('infer_result.csv',index=False)
        # break
end_time = time.time()
elapsed_time = end_time - start_time
print("Finished in {:.2f} seconds.".format(elapsed_time))

  0%|          | 0/9685 [00:00<?, ?it/s]/root/miniconda3/lib/python3.8/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
100%|██████████| 9685/9685 [3:35:23<00:00,  1.33s/it]  

Finished in 12923.70 seconds.
